# WebForms Example
----------------
The **WebForms** example automation allows you to create a web form. Users submit data through different field types (text field, number input, hidden input, dropdown choice, checkbox).

Upon submission, the automation processes the inputs and responds with JSON-formatted data.

##### 1. Import BitSwan libraries

In [ ]:
from bspump.http.web.server import *
from bspump.jupyter import *
import asyncio

#### 2. Create a sample event for testing
We start with a sample event so that we have something to work with. This event demonstrates the content submitted via the web form. In the example, you can see different values for different field types.


In [ ]:
event = {
    "form": {
        "subject":      "Test Subject",
        "description":  "This is a readonly description",
        "number":       42,
        "hiddenValue":  "hidden_value",
        "lol":          "b",
        "checkbox":     True,
        "date":         "2025-06-26",            
        "timestamp":    "2025-06-26T14:30:00"   
    }
}

##### 3. Define the webform
We define the web form. The definition of each field is as follows:

`TypeOfField("NameOfField", optional parameters)`

**TypeOfField**

Selects the field type. Available types:
- `DateField` .. date input 
- `DateTimeField` .. date and time input
- `TextField` .. text input
- `IntField` .. number input
- `ChoiceField` .. dropdown (user selects one option)
- `CheckboxField` .. checkbox

**NameOfField**

The name of the field used to access its value in code. See the Pipeline section below.

**Special parameters**

Each field can also include optional parameters. Available parameters:
- `display` .. Text (e.g., field label) shown above the field. If not provided, `NameOfField` is used. Example: `DateField("date", display="Date")`
- `readonly` .. When `readonly=True`, the field value cannot be modified by the user.
- `required` .. When `required=False`, the field is optional. If not specified, the field is required.
- `default` .. Default value of the field.
- `choices` .. **Only applicable** to `ChoiceField.` Defines the dropdown options. Example: `ChoiceField("letter", choices=["a", "b", "c"])`


In [ ]:
auto_pipeline(
    source=lambda app, pipeline: WebFormSource(app, pipeline, route="/", fields=[
        DateField("date", display="Date"),
        DateTimeField("timestamp", display="Date & Time"),  
        TextField("subject", display="foor"),
        TextField("description", readonly=True, default="laa"),
        IntField("number"),
        TextField("hiddenValue", hidden=True),
        ChoiceField("lol", choices=["a","b","c"]),
        CheckboxField("checkbox"),
    ]),
    sink  =lambda app, pipeline: WebSink(app,pipeline),
    name  ="WebServerPipeline"
)

#### 4. Pipeline section

Everything after this runs every time an event comes in. At runtime (once deployed), the `event` variable is automatically set to the value received from the source (in this case, the web form).

The form data is available at `event["form"]["NameOfField"]`.

You can apply any transformations you want, and then set `event["response"]` to define the form output. To return JSON, build `event["response"]` as a Python dictionary and run `json.dumps()` before sending it.

Example: `event["response"] = json.dumps({"number_times_five": event["form"]["number"] * 5})`

It is important to wrap the `event["response"]` block in a `try-except` clause (as shown below) so that if an error occurs (such as missing fields), the form returns an error message.

In [ ]:
# print the form content which is set up in the sample event
print(event["form"])

In [ ]:
# perform calculation
number_times_five = event["form"]["number"] * 5

In [ ]:
# set up the response
try:
    event["response"] = json.dumps({
    "msg":       event["form"]["subject"],
    "checkbox":  event["form"]["checkbox"],
    "number_times_five":    number_times_five,
    "lol":       event["form"]["lol"],
    "date":      event["form"]["date"],
    "timestamp": event["form"]["timestamp"],
    })
    event["content_type"] = "text/plain"
except e:
    event["response"] = {"error", str(e)}

##### 5. Testing

**Running the automation cell by cell**

To test the functionality before deployment, you can run the notebook cell by cell. The event used will be the sample event defined above.


In [ ]:
# Print the output of the automation run with sample event
print(event["response"])

**Running the automation in BitSwan Editor**

To run the automation in the BitSwan Editor, click **Deploy automation**. The form will be accessible via **Open External URL**. For further explanation of how the BitSwan Editor and automations work, please refer to the Development guide, accessible in the Editor.